In [2]:
# basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # graph

# importing sklearn datasets 
from sklearn import datasets 

# data pre-processing
from sklearn.preprocessing import StandardScaler # 연속변수 표준화 // scaler standardization (평균0 표편1)
from sklearn.preprocessing import LabelEncoder # 범주형(categorical) 변수 숫자형(Numerical)으로 변환해주는 package

# split train/test data 
from sklearn.model_selection import train_test_split # data split (train/test)

# classfication model 
from sklearn.naive_bayes import GaussianNB # 나이브 베이즈 분류
from sklearn.tree import DecisionTreeClassifier # 의사결정나무
from sklearn.ensemble import RandomForestClassifier # 랜덤 포레스트
from sklearn.ensemble import BaggingClassifier # 앙상블
from sklearn.linear_model import Perceptron # 퍼셉트론
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀분석
from sklearn.svm import SVC # 서포트 벡터머신
from sklearn.neural_network import MLPClassifier # 다층 인공신경망

# model test
from sklearn.metrics import confusion_matrix, classification_report # 정오분류
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer # 정확,예측,민감,f1스코어들
from sklearn.metrics import roc_curve # drawing ROC curve 

# optimization
from sklearn.model_selection import cross_validate # 교차타당성 
from sklearn.pipeline import make_pipeline # 파이프라인 구축 (scaler/classfication 한번에)
from sklearn.model_selection import learning_curve, validation_curve # (학습/검증 곡선)
from sklearn.model_selection import GridSearchCV # hyperparameter optimization

In [11]:
import glob
df = glob.glob('*csv*')
rm_df = pd.read_csv(df[0])
rm_df.tail()


,Income,Lot_Size,Ownership
43,68.0,18.4,Nonowner
44,49.4,16.4,Nonowner
45,35.0,18.8,Nonowner
46,53.0,14.0,Nonowner
47,65.0,14.8,Nonowner


In [9]:
rm_df.describe()

,Income,Lot_Size
count,48.000000,48.000000
mean,68.604167,18.950000
std,19.657828,2.402304
min,33.000000,14.000000
25%,52.950000,17.500000
50%,64.800000,19.000000
75%,83.125000,20.800000
max,112.100000,23.600000


In [10]:
rm_df.keys()

Index(['Income', 'Lot_Size', 'Ownership'], dtype='object')

In [26]:
X = rm_df.iloc[:,:2]
# X = rm_df.drop(['Ownership'],axis=1)
X.head()

y = rm_df.iloc[:,2]
y.head()

0    Owner
1    Owner
2    Owner
3    Owner
4    Owner
Name: Ownership, dtype: object

In [30]:
LE = LabelEncoder()
y_le = LE.fit_transform(y)
print(y_le)
# transform된 class 확인 방법
print(LE.classes_) # Nonowner : 0 , Owner : 1 

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
['Nonowner' 'Owner']


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)
# split 옵션
#  test_size비율 : 30%
#  random_seed : 1로 지정
#  stratify : y의 비율에 따라 분할

In [37]:
y_train.describe()

count           33
unique           2
top       Nonowner
freq            17
Name: Ownership, dtype: object

In [ ]:
# 모델 선택 및 구축
tree = DecisionTreeClassifier(criterion='gini',max_depth=1,random_state=1)
# criterion='gini',
# max_depth=1,
# random_state=1

# 모델 실행
tree.fit(X_train, y_train)